<div class="alert alert-block alert-info" align=center><b>Merchant Fraud Detection </b>

</div>

In [447]:
## Import all the modules
import numpy as np
import pandas as pd

import pandas_profiling as pprof
from datetime import datetime

import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


In [185]:
from sklearn import preprocessing
from sklearn.impute import SimpleImputer


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.compose import ColumnTransformer

In [304]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score,f1_score,confusion_matrix

import matplotlib.pyplot as plt 
import seaborn as sns

from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-info">
<br />
<b>Business Understanding</b>
<br /><br />
Fraud categories to check:<br />
1) orders from same ip / dev ids<br />
2) spike <br />
3) pattern in order<br />
4) same customers to specific merchants/group<br /> 

</div>

## Read the data

In [187]:
os.chdir('C:\\Users\\trex\\Downloads\\ML Comp Practice\\Mith')

merchant_data = pd.read_csv("train_merchant_data-1561627820784.csv", sep=',', header=0)
order_data =  pd.read_csv("train_order_data-1561627847149.csv", sep=',', header=0)
merchant_fraud = pd.read_csv("train-1561627878332.csv", sep=',', header=0)


print(merchant_data.shape)
print(order_data.shape)
print(merchant_fraud.shape)





(54213, 7)
(54213, 7)
(54213, 2)


In [188]:
## READ test data and check if there is any anamoly with train data

# Change to the test_data dir
import os
print(os.getcwd())
os.chdir('C:\\Users\\trex\\Downloads\\ML Comp Practice\\Mith\\test_data')
print(os.getcwd())


test_merchant_data = pd.read_csv("test_merchant_data-1561627903902.csv",sep=",",header=0)
test_order_data =  pd.read_csv("test_order_data-1561627931868.csv", sep=',', header=0)
test_merchant_fraud = pd.read_csv("test-1561627952093.csv", sep=',', header=0)


print(test_merchant_data.shape)
print(test_order_data.shape)
print(test_merchant_fraud.shape)

print(test_merchant_data.Merchant_ID.unique().shape)
print(test_order_data.Merchant_ID.unique().shape)
print(test_merchant_fraud.Merchant_ID.unique().shape)

# Here also test data : Merchant_ID is unique |


new_test_temp1 = pd.merge(test_merchant_data, test_order_data,  how='inner', left_on=['Merchant_ID'], right_on = ['Merchant_ID'])
print(new_test_temp1.shape)
print(new_test_temp1.columns.values)

# Columns are matching with train data | Hence no anamoly to be corrected


C:\Users\trex\Downloads\ML Comp Practice\Mith
C:\Users\trex\Downloads\ML Comp Practice\Mith\test_data
(13554, 7)
(13554, 7)
(13554, 1)
(13554,)
(13554,)
(13554,)
(13554, 13)
['Ecommerce_Provider_ID' 'Merchant_ID' 'Merchant_Registration_Date'
 'Registered_Device_ID' 'Gender' 'Age' 'IP_Address' 'Customer_ID'
 'Order_ID' 'Date_of_Order' 'Order_Value_USD' 'Order_Source'
 'Order_Payment_Method']


In [189]:
merchant_data.head()

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172


In [190]:
order_data.head()


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID
0,126221,37cea9512f8d,2018-04-29 16:39:26,148,Direct,Credit Card,124231
1,115471,09f12e6efde2,2018-06-16 17:05:40,145,SEO,Credit Card,136178
2,151786,4e69e956e159,2018-10-26 18:00:46,62,Ads,Internet Banking,198611
3,140456,663443aaeb82,2018-12-12 05:41:52,28,SEO,Debit Card,127993
4,114721,99258810c121,2018-09-20 11:06:10,70,Ads,Credit Card,250146


In [191]:
merchant_fraud.head()

,Merchant_ID,Fraudster
0,221592,0
1,316935,1
2,38454,1
3,214437,1
4,296240,1


### List of unique Merchant in company and the data given 

In [192]:
merchant_data.Merchant_ID.unique().shape

(54213,)

In [193]:
order_data.Merchant_ID.unique().shape

(54213,)

In [194]:
merchant_fraud.Merchant_ID.unique().shape

(54213,)

In [195]:
### No anamoly : all the merchant ids are same across the data given

In [196]:
### All are unique Merchants and there are no duplicate transactions by anyone
### No. of merchants = No. of transactions/data given
### so mostly they will have multiple Merchant_IDs for fraud

In [197]:
new_train_temp1 = pd.merge(merchant_data, order_data,  how='inner', left_on=['Merchant_ID'], right_on = ['Merchant_ID'])

In [198]:
new_train.shape

(54213, 28)

In [199]:
new_train = pd.merge(new_train_temp1,merchant_fraud , how='inner', left_on=['Merchant_ID'], right_on = ['Merchant_ID'])

In [200]:
new_train.shape

(54213, 14)

In [201]:
new_train.head()

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Fraudster
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,129697,3b8983a83c7b,2018-07-30 10:59:13,90,SEO,Debit Card,0
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,117390,34b5eb921228,2018-06-15 11:19:47,98,SEO,Internet Banking,0
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,120162,41a1c86ff08b,2018-08-13 10:06:26,95,SEO,Credit Card,0
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,128228,e8c3ad80d916,2018-07-22 15:46:51,100,Direct,E-wallet,0
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,136029,e71ab1f26785,2018-04-16 08:02:44,78,SEO,E-wallet,0


### Understand the data

In [202]:
print(new_train.columns.values)

['Ecommerce_Provider_ID' 'Merchant_ID' 'Merchant_Registration_Date'
 'Registered_Device_ID' 'Gender' 'Age' 'IP_Address' 'Customer_ID'
 'Order_ID' 'Date_of_Order' 'Order_Value_USD' 'Order_Source'
 'Order_Payment_Method' 'Fraudster']


In [203]:
print(new_train.info())

print("---------------------------------------------------------------------------------------------------------------------")
print(new_train.describe())
print("---------++++++++++------------------------+++++++++++-----------------------------------+++++------------------------")
print(new_train.describe(include=['object']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54213 entries, 0 to 54212
Data columns (total 14 columns):
Ecommerce_Provider_ID         54213 non-null int64
Merchant_ID                   54213 non-null int64
Merchant_Registration_Date    54213 non-null object
Registered_Device_ID          54213 non-null object
Gender                        54213 non-null object
Age                           54213 non-null int64
IP_Address                    54213 non-null object
Customer_ID                   54213 non-null int64
Order_ID                      54213 non-null object
Date_of_Order                 54213 non-null object
Order_Value_USD               54213 non-null int64
Order_Source                  54213 non-null object
Order_Payment_Method          54213 non-null object
Fraudster                     54213 non-null int64
dtypes: int64(6), object(8)
memory usage: 6.2+ MB
None
------------------------------------------------------------------------------------------------------------------

## Insights of data

In [204]:
pprof.ProfileReport(new_train)

Number of variables,14
Number of observations,54213
Total Missing (%),0.0%
Total size in memory,6.2 MiB
Average record size in memory,120.0 B
Numeric,4
Categorical,6
Boolean,1
Date,0
Text (Unique),2
Rejected,1


In [205]:
## Many IPs are repeated | 10+ may be
## IPs correlated with Dev-Ids



In [206]:
new_train.corr().iplot(kind='heatmap',colorscale="Blues",title="Feature Correlation Matrix")

In [207]:
# Not giving much info ...relation to Fraudster
# Age and OrderValue is stronlgly correlated


### Drawing trends towards dependent variable


In [208]:
new_train.groupby('Fraudster').mean()

,Ecommerce_Provider_ID,Merchant_ID,Age,Customer_ID,Order_Value_USD
Fraudster,,,,,
0,1746213.0,200348.491746,33.102118,137975.374421,92.213934
1,1746213.0,200852.154428,33.319204,137877.314030,92.389851


### Distribition of dependent variable


In [209]:
new_train["Fraudster"].iplot(kind="histogram", theme="white", title="Frauder vs Not",xTitle='Fraudsters', yTitle='Count')
new_train["Fraudster"].value_counts()

0    49188
1     5025
Name: Fraudster, dtype: int64

In [210]:
# What is the percentage?
fraud_yes = len(new_train[new_train.Fraudster == 1])
fraud_no = len(new_train[new_train.Fraudster!= 1])

percent_success = (fraud_yes/(fraud_yes + fraud_no))*100

print('Percentage of people who have done fraud:', np.round(percent_success,5), "%")

Percentage of people who have done fraud: 9.26899 %


In [211]:
## Since the distribution is skewed, Have to use SMOTE or enchanced sampling later



### Feature Engineering

In [212]:
new_train.head(20)

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Fraudster
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,129697,3b8983a83c7b,2018-07-30 10:59:13,90,SEO,Debit Card,0
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,117390,34b5eb921228,2018-06-15 11:19:47,98,SEO,Internet Banking,0
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,120162,41a1c86ff08b,2018-08-13 10:06:26,95,SEO,Credit Card,0
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,128228,e8c3ad80d916,2018-07-22 15:46:51,100,Direct,E-wallet,0
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,136029,e71ab1f26785,2018-04-16 08:02:44,78,SEO,E-wallet,0
5,1746213,51878,2018-07-29 05:40:41,YSUUQMHOIJZHJ,M,30,10.119.144.96,119105,942386c5ab2c,2018-10-11 09:41:51,75,Ads,Credit Card,0
6,1746213,205612,2018-02-03 05:37:07,VOQNWQHJUKFSZ,F,28,135.121.143.189,150927,f5135bb9ab13,2018-03-27 20:06:24,65,SEO,Credit Card,0
7,1746213,300823,2018-02-24 12:16:03,BQKBQIXSRMCHT,M,31,106.24.64.41,142294,d6f648eb802f,2018-04-12 02:19:59,152,SEO,Internet Banking,0
8,1746213,172822,2018-01-22 12:29:32,MKGAJACYBWCXO,F,27,160.16.202.244,155784,0f98e4cfd92d,2018-02-02 16:39:03,80,SEO,Debit Card,1
9,1746213,241476,2018-01-11 22:05:15,JJHGNMWRZRCML,M,41,93.33.235.39,138451,86ffa45527dc,2018-01-13 22:05:16,155,Ads,Credit Card,1


In [213]:
new_train.drop('Ecommerce_Provider_ID',axis=1,inplace=True)

In [214]:
## COPY of train data after dropping Provider ID | Raw format

new_train_bkup1 = new_train.copy(deep=True)

In [215]:
new_train.head(20)

,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Fraudster
0,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,129697,3b8983a83c7b,2018-07-30 10:59:13,90,SEO,Debit Card,0
1,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,117390,34b5eb921228,2018-06-15 11:19:47,98,SEO,Internet Banking,0
2,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,120162,41a1c86ff08b,2018-08-13 10:06:26,95,SEO,Credit Card,0
3,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,128228,e8c3ad80d916,2018-07-22 15:46:51,100,Direct,E-wallet,0
4,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,136029,e71ab1f26785,2018-04-16 08:02:44,78,SEO,E-wallet,0
5,51878,2018-07-29 05:40:41,YSUUQMHOIJZHJ,M,30,10.119.144.96,119105,942386c5ab2c,2018-10-11 09:41:51,75,Ads,Credit Card,0
6,205612,2018-02-03 05:37:07,VOQNWQHJUKFSZ,F,28,135.121.143.189,150927,f5135bb9ab13,2018-03-27 20:06:24,65,SEO,Credit Card,0
7,300823,2018-02-24 12:16:03,BQKBQIXSRMCHT,M,31,106.24.64.41,142294,d6f648eb802f,2018-04-12 02:19:59,152,SEO,Internet Banking,0
8,172822,2018-01-22 12:29:32,MKGAJACYBWCXO,F,27,160.16.202.244,155784,0f98e4cfd92d,2018-02-02 16:39:03,80,SEO,Debit Card,1
9,241476,2018-01-11 22:05:15,JJHGNMWRZRCML,M,41,93.33.235.39,138451,86ffa45527dc,2018-01-13 22:05:16,155,Ads,Credit Card,1


In [216]:
new_train.isnull().sum()

Merchant_ID                   0
Merchant_Registration_Date    0
Registered_Device_ID          0
Gender                        0
Age                           0
IP_Address                    0
Customer_ID                   0
Order_ID                      0
Date_of_Order                 0
Order_Value_USD               0
Order_Source                  0
Order_Payment_Method          0
Fraudster                     0
dtype: int64

In [217]:
new_train.IP_Address.duplicated().sum()

2185

In [218]:
new_train.Registered_Device_ID.duplicated().sum()

2922

In [219]:
new_train['is_dup_IP_Address']=new_train.IP_Address.duplicated()

In [220]:
new_train['is_dup_Device_ID']=new_train.Registered_Device_ID.duplicated()

In [221]:
new_train['is_dup_Device_ID'].sum()

2922

In [222]:
new_train['is_dup_IP_Address'].sum()

2185

In [223]:
pd.DataFrame(new_train,columns=['is_dup_IP_Address','IP_Address'])
new_train[new_train['is_dup_IP_Address']==True]

,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Fraudster,is_dup_IP_Address,is_dup_Device_ID
215,275990,2018-01-11 22:05:17,JJHGNMWRZRCML,M,41,93.33.235.39,135535,6d3c2cc5f800,2018-01-13 22:05:18,155,Ads,Credit Card,1,True,True
694,240172,2018-01-12 09:02:00,DIZIGMBVWQXAE,M,36,14.8.103.146,159422,9cf741c38145,2018-01-14 09:02:01,148,Direct,Internet Banking,1,True,True
719,196796,2018-01-06 13:32:54,QJHMAXGCKVSMV,F,50,145.74.25.201,125148,c6595cfeb8ed,2018-01-08 13:32:55,58,Ads,Credit Card,1,True,True
767,207003,2018-01-01 15:43:31,LWIYVWPWHUMGU,F,34,230.235.57.64,145123,329a89870c7e,2018-01-03 15:43:32,110,Direct,Internet Banking,1,True,True
977,338411,2018-01-06 04:06:24,NVAGSITHFTMSE,M,34,96.208.157.3,141251,4a928378f99b,2018-01-08 04:06:25,75,Direct,Debit Card,1,True,True
1471,213534,2018-01-01 15:43:29,LWIYVWPWHUMGU,F,34,230.235.57.64,125914,a0177d312c00,2018-01-19 16:56:05,110,Direct,Internet Banking,0,True,True
1653,177411,2018-01-05 13:34:12,WBGZNRTIFYVUH,M,31,94.244.25.161,136708,14cc97b50c1f,2018-01-07 13:34:13,65,Ads,Debit Card,1,True,True
1677,207602,2018-01-03 17:16:50,XQTIIJWXWJTFU,M,50,114.98.89.52,142639,de3a422ed18c,2018-01-05 17:16:51,102,Ads,Debit Card,1,True,True
2165,306264,2018-01-07 18:14:18,NFGJBLATZHTKW,M,32,255.24.130.186,137931,bd97d582f6ed,2018-01-09 18:14:19,102,Ads,Credit Card,1,True,True
2358,320229,2018-01-02 11:31:24,EKRFGOOSEMDPD,F,23,148.235.122.229,131757,8ed764b0cf35,2018-01-04 11:31:25,135,SEO,Credit Card,1,True,True


In [224]:
new_train.Customer_ID.unique().shape

(34081,)

In [225]:
new_train.shape

(54213, 15)

In [226]:
new_train.Customer_ID.duplicated().sum()

20132

In [227]:
new_train['is_duplicate_Customer_ID'] = new_train.Customer_ID.duplicated()

In [228]:
new_train['is_duplicate_Customer_ID'].sum()

20132

In [229]:
new_train

,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Fraudster,is_dup_IP_Address,is_dup_Device_ID,is_duplicate_Customer_ID
0,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,129697,3b8983a83c7b,2018-07-30 10:59:13,90,SEO,Debit Card,0,False,False,False
1,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,117390,34b5eb921228,2018-06-15 11:19:47,98,SEO,Internet Banking,0,False,False,False
2,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,120162,41a1c86ff08b,2018-08-13 10:06:26,95,SEO,Credit Card,0,False,False,False
3,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,128228,e8c3ad80d916,2018-07-22 15:46:51,100,Direct,E-wallet,0,False,False,False
4,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,136029,e71ab1f26785,2018-04-16 08:02:44,78,SEO,E-wallet,0,False,False,False
5,51878,2018-07-29 05:40:41,YSUUQMHOIJZHJ,M,30,10.119.144.96,119105,942386c5ab2c,2018-10-11 09:41:51,75,Ads,Credit Card,0,False,False,False
6,205612,2018-02-03 05:37:07,VOQNWQHJUKFSZ,F,28,135.121.143.189,150927,f5135bb9ab13,2018-03-27 20:06:24,65,SEO,Credit Card,0,False,False,False
7,300823,2018-02-24 12:16:03,BQKBQIXSRMCHT,M,31,106.24.64.41,142294,d6f648eb802f,2018-04-12 02:19:59,152,SEO,Internet Banking,0,False,False,False
8,172822,2018-01-22 12:29:32,MKGAJACYBWCXO,F,27,160.16.202.244,155784,0f98e4cfd92d,2018-02-02 16:39:03,80,SEO,Debit Card,1,False,False,False
9,241476,2018-01-11 22:05:15,JJHGNMWRZRCML,M,41,93.33.235.39,138451,86ffa45527dc,2018-01-13 22:05:16,155,Ads,Credit Card,1,False,False,False


In [230]:

merchant_data = datetime.strptime('2018-05-01 21:15:11', '%Y-%m-%d  %H:%M:%S')

In [231]:

new_train['new_formatted_date'] = pd.to_datetime(new_train.Merchant_Registration_Date)

In [232]:
new_train['new_formatted_date'].head()

0   2018-05-01 21:15:11
1   2018-04-14 10:13:00
2   2018-06-20 07:44:22
3   2018-06-27 01:41:39
4   2018-01-05 00:55:41
Name: new_formatted_date, dtype: datetime64[ns]

In [233]:
new_train['Merchant_Registration_Year'] = new_train.new_formatted_date.dt.strftime("%Y")

In [234]:
new_train['Merchant_Registration_Month'] =  new_train.new_formatted_date.dt.strftime("%m")

In [235]:
new_train['Merchant_Registration_Day'] =  new_train.new_formatted_date.dt.strftime("%d")

In [236]:
new_train['Merchant_Registration_Hour'] =  new_train.new_formatted_date.dt.strftime("%H")
new_train['Merchant_Registration_Min'] =  new_train.new_formatted_date.dt.strftime("%M")
new_train['Merchant_Registration_Sec'] =  new_train.new_formatted_date.dt.strftime("%S")

In [237]:
pd.DataFrame(new_train,columns=['Merchant_Registration_Date','Merchant_Registration_Year','Merchant_Registration_Month','Merchant_Registration_Day','Merchant_Registration_Hour','Merchant_Registration_Min','Merchant_Registration_Sec'])

,Merchant_Registration_Date,Merchant_Registration_Year,Merchant_Registration_Month,Merchant_Registration_Day,Merchant_Registration_Hour,Merchant_Registration_Min,Merchant_Registration_Sec
0,2018-05-01 21:15:11,2018,05,01,21,15,11
1,2018-04-14 10:13:00,2018,04,14,10,13,00
2,2018-06-20 07:44:22,2018,06,20,07,44,22
3,2018-06-27 01:41:39,2018,06,27,01,41,39
4,2018-01-05 00:55:41,2018,01,05,00,55,41
5,2018-07-29 05:40:41,2018,07,29,05,40,41
6,2018-02-03 05:37:07,2018,02,03,05,37,07
7,2018-02-24 12:16:03,2018,02,24,12,16,03
8,2018-01-22 12:29:32,2018,01,22,12,29,32
9,2018-01-11 22:05:15,2018,01,11,22,05,15


In [238]:
new_train.head()

,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,...,is_dup_IP_Address,is_dup_Device_ID,is_duplicate_Customer_ID,new_formatted_date,Merchant_Registration_Year,Merchant_Registration_Month,Merchant_Registration_Day,Merchant_Registration_Hour,Merchant_Registration_Min,Merchant_Registration_Sec
0,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,129697,3b8983a83c7b,2018-07-30 10:59:13,90,...,False,False,False,2018-05-01 21:15:11,2018,05,01,21,15,11
1,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,117390,34b5eb921228,2018-06-15 11:19:47,98,...,False,False,False,2018-04-14 10:13:00,2018,04,14,10,13,00
2,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,120162,41a1c86ff08b,2018-08-13 10:06:26,95,...,False,False,False,2018-06-20 07:44:22,2018,06,20,07,44,22
3,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,128228,e8c3ad80d916,2018-07-22 15:46:51,100,...,False,False,False,2018-06-27 01:41:39,2018,06,27,01,41,39
4,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,136029,e71ab1f26785,2018-04-16 08:02:44,78,...,False,False,False,2018-01-05 00:55:41,2018,01,05,00,55,41


In [239]:
new_train['new_formatted_date__order'] = pd.to_datetime(new_train.Date_of_Order)

In [240]:
new_train['new_formatted_date__order'].head()

0   2018-07-30 10:59:13
1   2018-06-15 11:19:47
2   2018-08-13 10:06:26
3   2018-07-22 15:46:51
4   2018-04-16 08:02:44
Name: new_formatted_date__order, dtype: datetime64[ns]

In [241]:
new_train['Order_Year'] = new_train.new_formatted_date__order.dt.strftime("%Y")
new_train['Order_Month'] =  new_train.new_formatted_date__order.dt.strftime("%m")
new_train['Order_Day'] =  new_train.new_formatted_date__order.dt.strftime("%d")
new_train['Order_Hour'] =  new_train.new_formatted_date__order.dt.strftime("%H")
new_train['Order_Min'] =  new_train.new_formatted_date__order.dt.strftime("%M")
new_train['Order_Sec'] =  new_train.new_formatted_date__order.dt.strftime("%S")


In [242]:
new_train.head()

,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,...,Merchant_Registration_Hour,Merchant_Registration_Min,Merchant_Registration_Sec,new_formatted_date__order,Order_Year,Order_Month,Order_Day,Order_Hour,Order_Min,Order_Sec
0,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,129697,3b8983a83c7b,2018-07-30 10:59:13,90,...,21,15,11,2018-07-30 10:59:13,2018,07,30,10,59,13
1,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,117390,34b5eb921228,2018-06-15 11:19:47,98,...,10,13,00,2018-06-15 11:19:47,2018,06,15,11,19,47
2,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,120162,41a1c86ff08b,2018-08-13 10:06:26,95,...,07,44,22,2018-08-13 10:06:26,2018,08,13,10,06,26
3,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,128228,e8c3ad80d916,2018-07-22 15:46:51,100,...,01,41,39,2018-07-22 15:46:51,2018,07,22,15,46,51
4,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,136029,e71ab1f26785,2018-04-16 08:02:44,78,...,00,55,41,2018-04-16 08:02:44,2018,04,16,08,02,44


In [243]:
pd.DataFrame(new_train,columns=['Date_of_Order','Order_Year','Order_Month','Order_Day','Order_Hour','Order_Min','Order_Sec']).head()

,Date_of_Order,Order_Year,Order_Month,Order_Day,Order_Hour,Order_Min,Order_Sec
0,2018-07-30 10:59:13,2018,07,30,10,59,13
1,2018-06-15 11:19:47,2018,06,15,11,19,47
2,2018-08-13 10:06:26,2018,08,13,10,06,26
3,2018-07-22 15:46:51,2018,07,22,15,46,51
4,2018-04-16 08:02:44,2018,04,16,08,02,44


In [244]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54213 entries, 0 to 54212
Data columns (total 30 columns):
Merchant_ID                    54213 non-null int64
Merchant_Registration_Date     54213 non-null object
Registered_Device_ID           54213 non-null object
Gender                         54213 non-null object
Age                            54213 non-null int64
IP_Address                     54213 non-null object
Customer_ID                    54213 non-null int64
Order_ID                       54213 non-null object
Date_of_Order                  54213 non-null object
Order_Value_USD                54213 non-null int64
Order_Source                   54213 non-null object
Order_Payment_Method           54213 non-null object
Fraudster                      54213 non-null int64
is_dup_IP_Address              54213 non-null bool
is_dup_Device_ID               54213 non-null bool
is_duplicate_Customer_ID       54213 non-null bool
new_formatted_date             54213 non-null datetime

In [245]:
import re

new_train['IP_Address_first']=(new_train['IP_Address'].str.rsplit(".",expand=True))[0]
new_train['IP_Address_second']=(new_train['IP_Address'].str.rsplit(".",expand=True))[1]
new_train['IP_Address_third']=(new_train['IP_Address'].str.rsplit(".",expand=True))[2]
new_train['IP_Address_fourth']=(new_train['IP_Address'].str.rsplit(".",expand=True))[3]

In [246]:
pd.DataFrame(new_train,columns=['IP_Address_first','IP_Address_second','IP_Address_third','IP_Address_fourth','IP_Address']).head()

,IP_Address_first,IP_Address_second,IP_Address_third,IP_Address_fourth,IP_Address
0,48,151,136,76,48.151.136.76
1,94,9,145,169,94.9.145.169
2,58,94,157,121,58.94.157.121
3,193,187,41,186,193.187.41.186
4,125,96,20,172,125.96.20.172


In [247]:
new_train_after_datetime_bkup = new_train.copy(deep=True)

In [248]:
print(new_train_after_datetime_bkup.shape)
print(new_train.shape)



(54213, 34)
(54213, 34)


In [249]:
new_train.head()

,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,...,Order_Year,Order_Month,Order_Day,Order_Hour,Order_Min,Order_Sec,IP_Address_first,IP_Address_second,IP_Address_third,IP_Address_fourth
0,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,129697,3b8983a83c7b,2018-07-30 10:59:13,90,...,2018,07,30,10,59,13,48,151,136,76
1,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,117390,34b5eb921228,2018-06-15 11:19:47,98,...,2018,06,15,11,19,47,94,9,145,169
2,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,120162,41a1c86ff08b,2018-08-13 10:06:26,95,...,2018,08,13,10,06,26,58,94,157,121
3,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,128228,e8c3ad80d916,2018-07-22 15:46:51,100,...,2018,07,22,15,46,51,193,187,41,186
4,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,136029,e71ab1f26785,2018-04-16 08:02:44,78,...,2018,04,16,08,02,44,125,96,20,172


In [250]:
cols_to_drop=['Merchant_ID','Merchant_Registration_Date','Registered_Device_ID','Order_ID','Date_of_Order',
              'new_formatted_date','new_formatted_date__order','IP_Address']

# have to get some info out of hashing of DeviceID or orderID if possible in next model !!!

new_train.drop(cols_to_drop,inplace=True,axis=1)

In [251]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54213 entries, 0 to 54212
Data columns (total 26 columns):
Gender                         54213 non-null object
Age                            54213 non-null int64
Customer_ID                    54213 non-null int64
Order_Value_USD                54213 non-null int64
Order_Source                   54213 non-null object
Order_Payment_Method           54213 non-null object
Fraudster                      54213 non-null int64
is_dup_IP_Address              54213 non-null bool
is_dup_Device_ID               54213 non-null bool
is_duplicate_Customer_ID       54213 non-null bool
Merchant_Registration_Year     54213 non-null object
Merchant_Registration_Month    54213 non-null object
Merchant_Registration_Day      54213 non-null object
Merchant_Registration_Hour     54213 non-null object
Merchant_Registration_Min      54213 non-null object
Merchant_Registration_Sec      54213 non-null object
Order_Year                     54213 non-null object


In [252]:
new_train.Order_Source.unique()

array(['SEO', 'Direct', 'Ads'], dtype=object)

In [253]:
new_test_temp1.Order_Source.unique()

array(['SEO', 'Ads', 'Direct'], dtype=object)

In [254]:
new_train.Order_Payment_Method.unique()

array(['Debit Card', 'Internet Banking', 'Credit Card', 'E-wallet',
       'Cash On Delivery'], dtype=object)

In [255]:
new_test_temp1.Order_Payment_Method.unique()

array(['Credit Card', 'Debit Card', 'E-wallet', 'Internet Banking',
       'Cash On Delivery'], dtype=object)

In [163]:
#No of Dummies categories  will be matching in Test and Train data; Hence we are safe


In [256]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54213 entries, 0 to 54212
Data columns (total 26 columns):
Gender                         54213 non-null object
Age                            54213 non-null int64
Customer_ID                    54213 non-null int64
Order_Value_USD                54213 non-null int64
Order_Source                   54213 non-null object
Order_Payment_Method           54213 non-null object
Fraudster                      54213 non-null int64
is_dup_IP_Address              54213 non-null bool
is_dup_Device_ID               54213 non-null bool
is_duplicate_Customer_ID       54213 non-null bool
Merchant_Registration_Year     54213 non-null object
Merchant_Registration_Month    54213 non-null object
Merchant_Registration_Day      54213 non-null object
Merchant_Registration_Hour     54213 non-null object
Merchant_Registration_Min      54213 non-null object
Merchant_Registration_Sec      54213 non-null object
Order_Year                     54213 non-null object


In [257]:
new_train_bkup_again=new_train.copy(deep=True)

In [450]:
## Adding extra columns for the mention of duplicated data : IP_Address, Device_ID, Customer_ID

In [259]:
new_train['is_dup_IP_Address_enc']=new_train['is_dup_IP_Address'].astype(int)

In [261]:
new_train.is_dup_IP_Address_enc.value_counts()

0    52028
1     2185
Name: is_dup_IP_Address_enc, dtype: int64

In [262]:
new_train.is_dup_IP_Address.value_counts()

False    52028
True      2185
Name: is_dup_IP_Address, dtype: int64

In [263]:
new_train['is_dup_Device_ID_enc']=new_train['is_dup_Device_ID'].astype(int)


In [264]:
new_train['is_duplicate_Customer_ID_enc']=new_train['is_duplicate_Customer_ID'].astype(int)

In [266]:
new_train.is_duplicate_Customer_ID_enc.value_counts()

0    34081
1    20132
Name: is_duplicate_Customer_ID_enc, dtype: int64

In [267]:
new_train.is_duplicate_Customer_ID.value_counts()

False    34081
True     20132
Name: is_duplicate_Customer_ID, dtype: int64

In [268]:
new_train.is_dup_Device_ID_enc.value_counts()

0    51291
1     2922
Name: is_dup_Device_ID_enc, dtype: int64

In [269]:
new_train.is_dup_Device_ID.value_counts()

False    51291
True      2922
Name: is_dup_Device_ID, dtype: int64

In [270]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54213 entries, 0 to 54212
Data columns (total 29 columns):
Gender                          54213 non-null object
Age                             54213 non-null int64
Customer_ID                     54213 non-null int64
Order_Value_USD                 54213 non-null int64
Order_Source                    54213 non-null object
Order_Payment_Method            54213 non-null object
Fraudster                       54213 non-null int64
is_dup_IP_Address               54213 non-null bool
is_dup_Device_ID                54213 non-null bool
is_duplicate_Customer_ID        54213 non-null bool
Merchant_Registration_Year      54213 non-null object
Merchant_Registration_Month     54213 non-null object
Merchant_Registration_Day       54213 non-null object
Merchant_Registration_Hour      54213 non-null object
Merchant_Registration_Min       54213 non-null object
Merchant_Registration_Sec       54213 non-null object
Order_Year                      54213

In [451]:
## Convert the data to necessary data types

In [271]:
cat_cols=['Gender','Order_Source','Order_Payment_Method']
num_cols=['Merchant_Registration_Year','Merchant_Registration_Month','Merchant_Registration_Day','Merchant_Registration_Hour',
         'Merchant_Registration_Min','Merchant_Registration_Sec','Order_Year','Order_Month','Order_Day','Order_Hour',
          'Order_Min','Order_Sec','IP_Address_first','IP_Address_second','IP_Address_third','IP_Address_fourth']

In [273]:
for item in cat_cols:
    new_train[item] = new_train[item].astype('category')

In [274]:
for item in num_cols:
    new_train[item] = new_train[item].astype('int64')

In [275]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54213 entries, 0 to 54212
Data columns (total 29 columns):
Gender                          54213 non-null category
Age                             54213 non-null int64
Customer_ID                     54213 non-null int64
Order_Value_USD                 54213 non-null int64
Order_Source                    54213 non-null category
Order_Payment_Method            54213 non-null category
Fraudster                       54213 non-null int64
is_dup_IP_Address               54213 non-null bool
is_dup_Device_ID                54213 non-null bool
is_duplicate_Customer_ID        54213 non-null bool
Merchant_Registration_Year      54213 non-null int64
Merchant_Registration_Month     54213 non-null int64
Merchant_Registration_Day       54213 non-null int64
Merchant_Registration_Hour      54213 non-null int64
Merchant_Registration_Min       54213 non-null int64
Merchant_Registration_Sec       54213 non-null int64
Order_Year                      54213

In [276]:
new_train.drop(['is_dup_IP_Address','is_dup_Device_ID','is_duplicate_Customer_ID'],inplace=True,axis=1)

In [277]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54213 entries, 0 to 54212
Data columns (total 26 columns):
Gender                          54213 non-null category
Age                             54213 non-null int64
Customer_ID                     54213 non-null int64
Order_Value_USD                 54213 non-null int64
Order_Source                    54213 non-null category
Order_Payment_Method            54213 non-null category
Fraudster                       54213 non-null int64
Merchant_Registration_Year      54213 non-null int64
Merchant_Registration_Month     54213 non-null int64
Merchant_Registration_Day       54213 non-null int64
Merchant_Registration_Hour      54213 non-null int64
Merchant_Registration_Min       54213 non-null int64
Merchant_Registration_Sec       54213 non-null int64
Order_Year                      54213 non-null int64
Order_Month                     54213 non-null int64
Order_Day                       54213 non-null int64
Order_Hour                      54

### Split into TEST and TRAIN

In [279]:
X = new_train.drop(["Fraudster"], axis = 1)

In [281]:
y = new_train["Fraudster"]

In [282]:
X.shape

(54213, 25)

In [283]:
y.shape

(54213,)

In [289]:
X.columns

Index(['Gender', 'Age', 'Customer_ID', 'Order_Value_USD', 'Order_Source',
       'Order_Payment_Method', 'Merchant_Registration_Year',
       'Merchant_Registration_Month', 'Merchant_Registration_Day',
       'Merchant_Registration_Hour', 'Merchant_Registration_Min',
       'Merchant_Registration_Sec', 'Order_Year', 'Order_Month', 'Order_Day',
       'Order_Hour', 'Order_Min', 'Order_Sec', 'IP_Address_first',
       'IP_Address_second', 'IP_Address_third', 'IP_Address_fourth',
       'is_dup_IP_Address_enc', 'is_dup_Device_ID_enc',
       'is_duplicate_Customer_ID_enc'],
      dtype='object')

In [284]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [285]:
train_objs_num = len(X_train)

In [286]:
train_objs_num

43370

In [287]:
dataset = pd.concat(objs=[X_train, X_test], axis=0)

In [288]:
dataset.shape

(54213, 25)

In [290]:
dataset.columns.values

array(['Gender', 'Age', 'Customer_ID', 'Order_Value_USD', 'Order_Source',
       'Order_Payment_Method', 'Merchant_Registration_Year',
       'Merchant_Registration_Month', 'Merchant_Registration_Day',
       'Merchant_Registration_Hour', 'Merchant_Registration_Min',
       'Merchant_Registration_Sec', 'Order_Year', 'Order_Month',
       'Order_Day', 'Order_Hour', 'Order_Min', 'Order_Sec',
       'IP_Address_first', 'IP_Address_second', 'IP_Address_third',
       'IP_Address_fourth', 'is_dup_IP_Address_enc',
       'is_dup_Device_ID_enc', 'is_duplicate_Customer_ID_enc'],
      dtype=object)

In [291]:
dataset_preprocessed = pd.get_dummies(dataset, columns=['Gender','Order_Source','Order_Payment_Method'],drop_first=True)

In [292]:
dataset_preprocessed.head()

,Age,Customer_ID,Order_Value_USD,Merchant_Registration_Year,Merchant_Registration_Month,Merchant_Registration_Day,Merchant_Registration_Hour,Merchant_Registration_Min,Merchant_Registration_Sec,Order_Year,...,is_dup_IP_Address_enc,is_dup_Device_ID_enc,is_duplicate_Customer_ID_enc,Gender_M,Order_Source_Direct,Order_Source_SEO,Order_Payment_Method_Credit Card,Order_Payment_Method_Debit Card,Order_Payment_Method_E-wallet,Order_Payment_Method_Internet Banking
52623,30,162257,138,2018,8,3,11,20,52,2018,...,0,0,0,1,1,0,0,1,0,0
49535,33,145790,80,2018,8,1,19,35,32,2018,...,0,1,0,1,0,1,0,1,0,0
48983,31,151697,108,2018,1,28,23,54,2,2018,...,0,0,1,0,1,0,1,0,0,0
51418,45,137049,128,2018,2,27,15,21,53,2018,...,0,0,1,1,0,1,1,0,0,0
42629,36,120517,78,2018,1,30,4,35,19,2018,...,0,0,0,0,1,0,0,0,0,1


In [293]:
X_train = dataset_preprocessed[:train_objs_num]

In [294]:
X_test = dataset_preprocessed[train_objs_num:]

In [295]:
print(X_train.shape)
print(X_test.shape)

(43370, 29)
(10843, 29)


### Implementing Models

### Logistic Model

In [296]:
logisticRegr = LogisticRegression()

In [297]:
logisticRegr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [298]:
logisticRegr.coef_

array([[ 4.36684757e-03, -1.23006988e-06, -1.83066284e-03,
        -1.52169576e-05, -3.66321576e-02, -5.89920728e-02,
         3.38873671e-03, -9.77111554e-04, -1.85361976e-03,
        -1.52169576e-05, -5.96221932e-02, -4.89946450e-02,
         1.41881374e-03,  5.54054837e-04, -6.69628855e-04,
        -2.58724481e-04, -4.65653164e-04, -2.15538988e-05,
         2.08402934e-04,  8.79996524e-03,  9.33773807e-03,
         3.52477403e-05,  2.38058488e-04,  8.08171696e-04,
        -6.58421625e-04,  7.28877903e-04, -8.91692260e-05,
        -1.45333545e-04, -4.20090494e-04]])

In [299]:
logisticRegr.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)>

In [300]:
logisticRegr.classes_

array([0, 1], dtype=int64)

In [301]:
logisticRegr.decision_function

<bound method LinearClassifierMixin.decision_function of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)>

In [302]:
train_predicted_classes = logisticRegr.predict(X_train)

In [305]:

train_accuracy = accuracy_score(y_train,train_predicted_classes)

In [306]:
train_accuracy

0.9071708554300207

In [307]:
print("TRAIN DATA ACCURACY",accuracy_score(y_train,train_predicted_classes))
print("\nTrain data f1-score for class '0'",f1_score(y_train,train_predicted_classes,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(y_train,train_predicted_classes,pos_label=1))

TRAIN DATA ACCURACY 0.9071708554300207

Train data f1-score for class '0' 0.9513262567400923

Train data f1-score for class '1' 0.0


In [320]:
print("\nTrain data f1-score",f1_score(y_train,train_predicted_classes))
print("\nTest data f1-score",f1_score(y_test,test_predicted_classes))


Train data f1-score 0.0

Test data f1-score 0.0


In [309]:
test_predicted_classes = logisticRegr.predict(X_test)

In [310]:
print("TEST DATA ACCURACY",accuracy_score(y_test,test_predicted_classes))
print("\nTest data f1-score for class '0'",f1_score(y_test,test_predicted_classes,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,test_predicted_classes,pos_label=1))

TEST DATA ACCURACY 0.9078668265240247

Test data f1-score for class '0' 0.9517088026296708

Test data f1-score for class '1' 0.0


## Decision Tree

In [325]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

dtc = DecisionTreeClassifier()

param_grid = {"max_depth" : [10, 12, 14, 16, 18, 20],
              "min_samples_leaf" : [5, 10, 15, 20],
              "class_weight" : ['balanced'],
              "min_samples_split": [2, 10, 20],
              "criterion": ['entropy', 'gini']}
 
dtc_cv_grid = RandomizedSearchCV(estimator = dtc, param_distributions = param_grid, cv = 3, n_iter=10)

In [326]:
dtc_cv_grid.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'max_depth': [10, 12, 14, 16, 18, 20], 'min_samples_leaf': [5, 10, 15, 20], 'class_weight': ['balanced'], 'min_samples_split': [2, 10, 20], 'criterion': ['entropy', 'gini']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [331]:
dtc_cv_grid.best_params_

{'min_samples_split': 10,
 'min_samples_leaf': 15,
 'max_depth': 10,
 'criterion': 'entropy',
 'class_weight': 'balanced'}

In [327]:
train_predicted_classes_dt = dtc_cv_grid.predict(X_train)

In [328]:
test_predicted_classes_dt = dtc_cv_grid.predict(X_test)

In [329]:
print("TRAIN DATA ACCURACY",accuracy_score(y_train,train_predicted_classes_dt))
print("\nTrain data f1-score for class '0'",f1_score(y_train,train_predicted_classes_dt,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(y_train,train_predicted_classes_dt,pos_label=1))
print("\nTrain data f1-score",f1_score(y_train,train_predicted_classes_dt))

TRAIN DATA ACCURACY 0.9373991238183076

Train data f1-score for class '0' 0.9657719897631145

Train data f1-score for class '1' 0.6340477153255155

Train data f1-score 0.6340477153255155


In [330]:
print("TEST DATA ACCURACY",accuracy_score(y_test,test_predicted_classes_dt))
print("\nTest data f1-score for class '0'",f1_score(y_test,test_predicted_classes_dt,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,test_predicted_classes_dt,pos_label=1))
print("\nTest data f1-score",f1_score(y_test,test_predicted_classes_dt))

TEST DATA ACCURACY 0.93230655722586

Test data f1-score for class '0' 0.9629442649434571

Test data f1-score for class '1' 0.6091586794462193

Test data f1-score 0.6091586794462193


## Random Forest


In [338]:
rfe = RandomForestClassifier()

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=143)


In [339]:
dt_param_grid = {"n_estimators" : [100],
              "max_depth" : [5,8,10],
              "max_features" : [3, 5, 7],
              "min_samples_leaf" : [4, 6, 8, 10]}
rf_grid = GridSearchCV(estimator = rfe, param_grid=dt_param_grid, cv=kfold)

In [340]:
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=143, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100], 'max_depth': [5, 8, 10], 'max_features': [3, 5, 7], 'min_samples_leaf': [4, 6, 8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [342]:
rf_grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [343]:
rf_grid.best_params_

{'max_depth': 10,
 'max_features': 3,
 'min_samples_leaf': 4,
 'n_estimators': 100}

In [344]:
## Taking too much time, so reduced the hypermaters to check

In [345]:
train_predicted_classes_rf = rf_grid.predict(X_train)
test_predicted_classes_rf = rf_grid.predict(X_test)

In [346]:
print("TRAIN DATA ACCURACY",accuracy_score(y_train,train_predicted_classes_rf))
print("\nTrain data f1-score for class '0'",f1_score(y_train,train_predicted_classes_rf,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(y_train,train_predicted_classes_rf,pos_label=1))
print("\nTrain data f1-score",f1_score(y_train,train_predicted_classes_rf))

TRAIN DATA ACCURACY 0.9551763892091307

Train data f1-score for class '0' 0.9758755056960611

Train data f1-score for class '1' 0.6843130886651511

Train data f1-score 0.6843130886651511


In [347]:
print("TEST DATA ACCURACY",accuracy_score(y_test,test_predicted_classes_rf))
print("\nTest data f1-score for class '0'",f1_score(y_test,test_predicted_classes_rf,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,test_predicted_classes_rf,pos_label=1))
print("\nTest data f1-score",f1_score(y_test,test_predicted_classes_rf))

TEST DATA ACCURACY 0.9558240339389468

Test data f1-score for class '0' 0.9762034875055889

Test data f1-score for class '1' 0.6923570969813745

Test data f1-score 0.6923570969813745


## SMOTE use

In [369]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43370 entries, 52623 to 52734
Data columns (total 29 columns):
Age                                      43370 non-null int64
Customer_ID                              43370 non-null int64
Order_Value_USD                          43370 non-null int64
Merchant_Registration_Year               43370 non-null int64
Merchant_Registration_Month              43370 non-null int64
Merchant_Registration_Day                43370 non-null int64
Merchant_Registration_Hour               43370 non-null int64
Merchant_Registration_Min                43370 non-null int64
Merchant_Registration_Sec                43370 non-null int64
Order_Year                               43370 non-null int64
Order_Month                              43370 non-null int64
Order_Day                                43370 non-null int64
Order_Hour                               43370 non-null int64
Order_Min                                43370 non-null int64
Order_Sec          

In [349]:
smote = SMOTE(random_state=0)

os_data_X,os_data_y=smote.fit_sample(X_train, y_train)

In [350]:
os_data_X = pd.DataFrame(data=os_data_X)
os_data_y= pd.DataFrame(data=os_data_y,columns=['Fraudster'])

In [351]:
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no fraudster in oversampled data",len(os_data_y[os_data_y['Fraudster']==0]))
print("Number of fraudster",len(os_data_y[os_data_y['Fraudster']==1]))
print("Proportion of no fraudster data in oversampled data is ",len(os_data_y[os_data_y['Fraudster']==0])/len(os_data_X))
print("Proportion of fraudster data in oversampled data is ",len(os_data_y[os_data_y['Fraudster']==1])/len(os_data_X))

length of oversampled data is  78688
Number of no fraudster in oversampled data 39344
Number of fraudster 39344
Proportion of no fraudster data in oversampled data is  0.5
Proportion of fraudster data in oversampled data is  0.5


In [352]:
sm_dtc = DecisionTreeClassifier()

param_grid = {"max_depth" : [10, 12, 14, 16, 18, 20],
              "min_samples_leaf" : [5, 10, 15, 20],
              "class_weight" : ['balanced'],
              "min_samples_split": [2, 10, 20],
              "criterion": ['entropy', 'gini']}
 
sm_dtc_cv_grid = RandomizedSearchCV(estimator = sm_dtc, param_distributions = param_grid, cv = 3, n_iter=10)

In [353]:

sm_dtc_cv_grid.fit(os_data_X,os_data_y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'max_depth': [10, 12, 14, 16, 18, 20], 'min_samples_leaf': [5, 10, 15, 20], 'class_weight': ['balanced'], 'min_samples_split': [2, 10, 20], 'criterion': ['entropy', 'gini']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78688 entries, 0 to 78687
Data columns (total 29 columns):
0     78688 non-null int64
1     78688 non-null int64
2     78688 non-null int64
3     78688 non-null int64
4     78688 non-null int64
5     78688 non-null int64
6     78688 non-null int64
7     78688 non-null int64
8     78688 non-null int64
9     78688 non-null int64
10    78688 non-null int64
11    78688 non-null int64
12    78688 non-null int64
13    78688 non-null int64
14    78688 non-null int64
15    78688 non-null int64
16    78688 non-null int64
17    78688 non-null int64
18    78688 non-null int64
19    78688 non-null int64
20    78688 non-null int64
21    78688 non-null int64
22    78688 non-null int64
23    78688 non-null int64
24    78688 non-null int64
25    78688 non-null int64
26    78688 non-null int64
27    78688 non-null int64
28    78688 non-null int64
dtypes: int64(29)
memory usage: 17.4 MB


In [354]:
sm_dtc_cv_grid.best_params_

{'min_samples_split': 20,
 'min_samples_leaf': 20,
 'max_depth': 18,
 'criterion': 'gini',
 'class_weight': 'balanced'}

In [355]:
sm_train_predicted_classes_dt = sm_dtc_cv_grid.predict(os_data_X)
sm_test_predicted_classes_dt = sm_dtc_cv_grid.predict(X_test)

In [356]:
print("TRAIN DATA ACCURACY",accuracy_score(os_data_y,sm_train_predicted_classes_dt))
print("\nTrain data f1-score for class '0'",f1_score(os_data_y,sm_train_predicted_classes_dt,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(os_data_y,sm_train_predicted_classes_dt,pos_label=1))
print("\nTrain data f1-score",f1_score(os_data_y,sm_train_predicted_classes_dt))

TRAIN DATA ACCURACY 0.9194413379422529

Train data f1-score for class '0' 0.9210260754731086

Train data f1-score for class '1' 0.9177916974672217

Train data f1-score 0.9177916974672217


In [358]:
print("TEST DATA ACCURACY",accuracy_score(y_test,sm_test_predicted_classes_dt))
print("\nTest data f1-score for class '0'",f1_score(y_test,sm_test_predicted_classes_dt,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,sm_test_predicted_classes_dt,pos_label=1))
print("\nTest data f1-score",f1_score(y_test,sm_test_predicted_classes_dt))

TEST DATA ACCURACY 0.886193857788435

Test data f1-score for class '0' 0.9360688011605015

Test data f1-score for class '1' 0.4823825503355705

Test data f1-score 0.4823825503355705


## XGBOOST model

<div class="alert alert-block alert-success" align=center> <b>Xgboost</b> was the best model </div>

In [379]:
from xgboost import XGBClassifier
XGB = XGBClassifier(n_jobs=-1)
 
# Use a grid over parameters of interest
param_grid = {
     'colsample_bytree': np.linspace(0.5, 0.9, 5),
     'n_estimators':[100],
     'max_depth': [10, 15, 20, 25],
    'class_weight':['balanced']
}

 
CV_XGB = GridSearchCV(estimator=XGB, param_grid=param_grid, cv= 3)

In [380]:
CV_XGB.fit(X = X_train, y=y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10, 15, 20, 25], 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [381]:
print (CV_XGB.best_score_, CV_XGB.best_params_, CV_XGB.best_estimator_)

0.9546460687110906 {'class_weight': 'balanced', 'colsample_bytree': 0.6, 'max_depth': 25, 'n_estimators': 100} XGBClassifier(base_score=0.5, booster='gbtree', class_weight='balanced',
       colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.6,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=25,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)


In [382]:
xg_train_predicted_classes_dt = CV_XGB.predict(X_train)
xg_test_predicted_classes_dt = CV_XGB.predict(X_test)

In [383]:
print("TRAIN DATA ACCURACY",accuracy_score(y_train,xg_train_predicted_classes_dt ))
print("\nTrain data f1-score for class '0'",f1_score(y_train,xg_train_predicted_classes_dt,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(y_train,xg_train_predicted_classes_dt,pos_label=1))
print("\nTrain data f1-score",f1_score(y_train,xg_train_predicted_classes_dt))

TRAIN DATA ACCURACY 1.0

Train data f1-score for class '0' 1.0

Train data f1-score for class '1' 1.0

Train data f1-score 1.0


In [384]:
print("TEST DATA ACCURACY",accuracy_score(y_test,xg_test_predicted_classes_dt))
print("\nTest data f1-score for class '0'",f1_score(y_test,xg_test_predicted_classes_dt,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,xg_test_predicted_classes_dt,pos_label=1))
print("\nTest data f1-score",f1_score(y_test,xg_test_predicted_classes_dt))

TEST DATA ACCURACY 0.9566540625288205

Test data f1-score for class '0' 0.9766564021058907

Test data f1-score for class '1' 0.6971649484536083

Test data f1-score 0.6971649484536083





### Test data feature engineering

In [417]:
# repeat the same pre-processing of train on test 

In [376]:
new_test_temp1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13554 entries, 0 to 13553
Data columns (total 13 columns):
Ecommerce_Provider_ID         13554 non-null int64
Merchant_ID                   13554 non-null int64
Merchant_Registration_Date    13554 non-null object
Registered_Device_ID          13554 non-null object
Gender                        13554 non-null object
Age                           13554 non-null int64
IP_Address                    13554 non-null object
Customer_ID                   13554 non-null int64
Order_ID                      13554 non-null object
Date_of_Order                 13554 non-null object
Order_Value_USD               13554 non-null int64
Order_Source                  13554 non-null object
Order_Payment_Method          13554 non-null object
dtypes: int64(5), object(8)
memory usage: 1.4+ MB


In [377]:
new_test_temp1.drop('Ecommerce_Provider_ID',axis=1,inplace=True)

In [386]:
new_test_temp1['is_dup_IP_Address']=new_test_temp1.IP_Address.duplicated()

new_test_temp1['is_dup_Device_ID']=new_test_temp1.Registered_Device_ID.duplicated()

new_test_temp1['is_duplicate_Customer_ID'] = new_test_temp1.Customer_ID.duplicated()

new_test_temp1['new_formatted_date'] = pd.to_datetime(new_test_temp1.Merchant_Registration_Date)

new_test_temp1['Merchant_Registration_Year'] = new_test_temp1.new_formatted_date.dt.strftime("%Y")

new_test_temp1['Merchant_Registration_Month'] =  new_test_temp1.new_formatted_date.dt.strftime("%m")

new_test_temp1['Merchant_Registration_Day'] =  new_test_temp1.new_formatted_date.dt.strftime("%d")

new_test_temp1['Merchant_Registration_Hour'] =  new_test_temp1.new_formatted_date.dt.strftime("%H")
new_test_temp1['Merchant_Registration_Min'] =  new_test_temp1.new_formatted_date.dt.strftime("%M")
new_test_temp1['Merchant_Registration_Sec'] =  new_test_temp1.new_formatted_date.dt.strftime("%S")

In [387]:
new_test_temp1['new_formatted_date__order'] = pd.to_datetime(new_test_temp1.Date_of_Order)
new_test_temp1['Order_Year'] = new_test_temp1.new_formatted_date__order.dt.strftime("%Y")
new_test_temp1['Order_Month'] =  new_test_temp1.new_formatted_date__order.dt.strftime("%m")
new_test_temp1['Order_Day'] =  new_test_temp1.new_formatted_date__order.dt.strftime("%d")
new_test_temp1['Order_Hour'] =  new_test_temp1.new_formatted_date__order.dt.strftime("%H")
new_test_temp1['Order_Min'] =  new_test_temp1.new_formatted_date__order.dt.strftime("%M")
new_test_temp1['Order_Sec'] =  new_test_temp1.new_formatted_date__order.dt.strftime("%S")


import re

new_test_temp1['IP_Address_first']=(new_test_temp1['IP_Address'].str.rsplit(".",expand=True))[0]
new_test_temp1['IP_Address_second']=(new_test_temp1['IP_Address'].str.rsplit(".",expand=True))[1]
new_test_temp1['IP_Address_third']=(new_test_temp1['IP_Address'].str.rsplit(".",expand=True))[2]
new_test_temp1['IP_Address_fourth']=(new_test_temp1['IP_Address'].str.rsplit(".",expand=True))[3]

cols_to_drop=['Merchant_ID','Merchant_Registration_Date','Registered_Device_ID','Order_ID','Date_of_Order',
              'new_formatted_date','new_formatted_date__order','IP_Address']

# have to get some info out of hashing of DeviceID or orderID if possible in next model !!!

new_test_temp1.drop(cols_to_drop,inplace=True,axis=1)

new_test_temp1['is_dup_IP_Address_enc']=new_test_temp1['is_dup_IP_Address'].astype(int)

new_test_temp1['is_dup_Device_ID_enc']=new_test_temp1['is_dup_Device_ID'].astype(int)

new_test_temp1['is_duplicate_Customer_ID_enc']=new_test_temp1['is_duplicate_Customer_ID'].astype(int)

In [388]:

cat_cols=['Gender','Order_Source','Order_Payment_Method']
num_cols=['Merchant_Registration_Year','Merchant_Registration_Month','Merchant_Registration_Day','Merchant_Registration_Hour',
         'Merchant_Registration_Min','Merchant_Registration_Sec','Order_Year','Order_Month','Order_Day','Order_Hour',
          'Order_Min','Order_Sec','IP_Address_first','IP_Address_second','IP_Address_third','IP_Address_fourth']


for item in cat_cols:
    new_test_temp1[item] = new_test_temp1[item].astype('category')


for item in num_cols:
    new_test_temp1[item] = new_test_temp1[item].astype('int64')

new_test_temp1.drop(['is_dup_IP_Address','is_dup_Device_ID','is_duplicate_Customer_ID'],inplace=True,axis=1)

In [389]:
new_test_temp1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13554 entries, 0 to 13553
Data columns (total 25 columns):
Gender                          13554 non-null category
Age                             13554 non-null int64
Customer_ID                     13554 non-null int64
Order_Value_USD                 13554 non-null int64
Order_Source                    13554 non-null category
Order_Payment_Method            13554 non-null category
Merchant_Registration_Year      13554 non-null int64
Merchant_Registration_Month     13554 non-null int64
Merchant_Registration_Day       13554 non-null int64
Merchant_Registration_Hour      13554 non-null int64
Merchant_Registration_Min       13554 non-null int64
Merchant_Registration_Sec       13554 non-null int64
Order_Year                      13554 non-null int64
Order_Month                     13554 non-null int64
Order_Day                       13554 non-null int64
Order_Hour                      13554 non-null int64
Order_Min                       13

In [393]:
 new_test_final = pd.get_dummies(new_test_temp1, columns=['Gender','Order_Source','Order_Payment_Method'],drop_first=True)

In [394]:
new_test_final.shape

(13554, 29)

In [397]:
X_train.shape

(43370, 29)

### Model on final test data

In [401]:
# predict on final test data
new_test_final.head()


,Age,Customer_ID,Order_Value_USD,Merchant_Registration_Year,Merchant_Registration_Month,Merchant_Registration_Day,Merchant_Registration_Hour,Merchant_Registration_Min,Merchant_Registration_Sec,Order_Year,...,is_dup_IP_Address_enc,is_dup_Device_ID_enc,is_duplicate_Customer_ID_enc,Gender_M,Order_Source_Direct,Order_Source_SEO,Order_Payment_Method_Credit Card,Order_Payment_Method_Debit Card,Order_Payment_Method_E-wallet,Order_Payment_Method_Internet Banking
0,40,157068,60,2018,2,11,20,50,29,2018,...,0,0,0,0,0,1,1,0,0,0
1,39,112534,28,2018,4,6,13,19,39,2018,...,0,0,0,1,0,1,1,0,0,0
2,23,148774,75,2018,1,12,16,47,49,2018,...,0,0,0,1,0,0,0,1,0,0
3,34,114528,98,2018,2,19,21,58,52,2018,...,0,0,0,1,0,1,1,0,0,0
4,20,120940,35,2018,5,17,15,50,19,2018,...,0,0,0,1,0,1,1,0,0,0


In [402]:
final_out_xg_ver1=CV_XGB.predict(new_test_final)

In [406]:
pd.DataFrame(final_out_xg_ver1).to_csv("file_xg_ver1.csv")

In [407]:
## After ver-1 , trying stacking !

In [409]:
X_train_KNN = X_train.copy(deep=True)

X_test_KNN = X_test.copy(deep=True)


scaler = StandardScaler()
scaler.fit(X_train_KNN)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [413]:
X_train_KNN=scaler.transform(X_train_KNN)
X_test_KNN=scaler.transform(X_test_KNN)

In [415]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

parameters = {'n_neighbors':list(range(5,10))}
clf = GridSearchCV(KNeighborsClassifier(), parameters, n_jobs=-1,verbose=2, cv=3)
clf.fit(X=X_train_KNN, y=y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  2.3min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_neighbors': [5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [420]:
knn_pred_train=clf.predict(X_train_KNN)
knn_pred_test = clf.predict(X_test_KNN)

In [421]:
print("TRAIN DATA ACCURACY",accuracy_score(y_train,knn_pred_train ))
print("\nTrain data f1-score for class '0'",f1_score(y_train,knn_pred_train,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(y_train,knn_pred_train,pos_label=1))
print("\nTrain data f1-score",f1_score(y_train,knn_pred_train))

TRAIN DATA ACCURACY 0.9492045192529398

Train data f1-score for class '0' 0.9726529041548221

Train data f1-score for class '1' 0.64370046902798

Train data f1-score 0.64370046902798


In [422]:
print("TEST DATA ACCURACY",accuracy_score(y_test,knn_pred_test))
print("\nTest data f1-score for class '0'",f1_score(y_test,knn_pred_test,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,knn_pred_test,pos_label=1))
print("\nTest data f1-score",f1_score(y_test,knn_pred_test))

TEST DATA ACCURACY 0.9489071290233331

Test data f1-score for class '0' 0.9724816212994237

Test data f1-score for class '1' 0.6435006435006436

Test data f1-score 0.6435006435006436


In [423]:
final_out_knn_ver2=clf.predict(new_test_final)

In [424]:
pd.DataFrame(final_out_knn_ver2).to_csv("file_knn_ver2.csv")

## Stacking

In [437]:
stack_train = pd.DataFrame([knn_pred_train,xg_train_predicted_classes_dt,train_predicted_classes_rf])

In [438]:
stack_test = pd.DataFrame([knn_pred_test,xg_test_predicted_classes_dt,test_predicted_classes_rf])

In [439]:
stack_train = stack_train.T
stack_test = stack_test.T

In [440]:
stack_train.columns = ['knn_pred_train','xg_train_predicted_classes_dt','train_predicted_classes_rf']
stack_test.columns = ['knn_pred_test','xg_test_predicted_classes_dt','test_predicted_classes_rf']

In [441]:
from sklearn.tree import DecisionTreeClassifier
dtc_stk = DecisionTreeClassifier()

dtc_stk.fit(stack_train,y_train)

stacked_pred_train = dtc_stk.predict(stack_train)
stacked_pred_test = dtc_stk.predict(stack_test)

In [442]:
print("TRAIN DATA ACCURACY",accuracy_score(y_train,stacked_pred_train))
print("\nTrain data f1-score for class '0'",f1_score(y_train,stacked_pred_train,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(y_train,stacked_pred_train,pos_label=1))

### Test data accuracy
print("\n\n--------------------------------------\n\n")
print("TEST DATA ACCURACY",accuracy_score(y_test,stacked_pred_test))
print("\nTest data f1-score for class '0'",f1_score(y_test,stacked_pred_test,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,stacked_pred_test,pos_label=1))

TRAIN DATA ACCURACY 1.0

Train data f1-score for class '0' 1.0

Train data f1-score for class '1' 1.0


--------------------------------------


TEST DATA ACCURACY 0.9566540625288205

Test data f1-score for class '0' 0.9766564021058907

Test data f1-score for class '1' 0.6971649484536083


In [446]:
# Run all models on Test data and use : KNN, RF, Xgboost
# and Take the mode() of the predictions for stacking

<div class="alert alert-block alert-success" align=center> ** END for Today!**  </div>

In [452]:
final_out_knn_ver2.sum()

0

In [454]:
final_out_xg_ver1.sum()

713

In [455]:
dataset_for_checking = pd.concat(objs=[X_train, X_test], axis=0)

In [456]:
dataset_out_for_checking=pd.concat(objs=[y_train,y_test],axis=0)

In [457]:
from xgboost import XGBClassifier
XGB = XGBClassifier(n_jobs=-1)
 
# Use a grid over parameters of interest
param_grid = {
     'colsample_bytree': np.linspace(0.5, 0.9, 5),
     'n_estimators':[100],
     'max_depth': [10, 15, 20, 25],
    'class_weight':['balanced']
}

 
CV_XGB_test = GridSearchCV(estimator=XGB, param_grid=param_grid, cv= 3)

In [458]:
CV_XGB_test.fit(dataset_for_checking,dataset_out_for_checking)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10, 15, 20, 25], 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [459]:
test_train_1=CV_XGB_test.predict(X_train)

In [460]:
test_test_1=CV_XGB_test.predict(X_test)

In [461]:
print("TRAIN DATA ACCURACY",accuracy_score(y_train,test_train_1 ))
print("\nTrain data f1-score for class '0'",f1_score(y_train,test_train_1,pos_label=0))
print("\nTrain data f1-score for class '1'",f1_score(y_train,test_train_1,pos_label=1))
print("\nTrain data f1-score",f1_score(y_train,test_train_1))

TRAIN DATA ACCURACY 1.0

Train data f1-score for class '0' 1.0

Train data f1-score for class '1' 1.0

Train data f1-score 1.0


In [462]:
print("TEST DATA ACCURACY",accuracy_score(y_test,test_test_1))
print("\nTest data f1-score for class '0'",f1_score(y_test,test_test_1,pos_label=0))
print("\nTest data f1-score for class '1'",f1_score(y_test,test_test_1,pos_label=1))
print("\nTest data f1-score",f1_score(y_test,test_test_1))

TEST DATA ACCURACY 1.0

Test data f1-score for class '0' 1.0

Test data f1-score for class '1' 1.0

Test data f1-score 1.0


In [463]:
full_xgb_on_test=CV_XGB_test.predict(new_test_final)

In [464]:
full_xgb_on_test.sum()

711